[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/flaviagiammarino/nbeats-tensorflow/tutorial/nbeats-tensorflow-tutorial.ipynb)

# N-BEATS TensorFlow Tutorial

**Paper:** Oreshkin, B. N., Carpov, D., Chapados, N., & Bengio, Y. (2019). N-BEATS: Neural basis expansion analysis for interpretable time series forecasting. [arXiv:1905.10437](https://arxiv.org/abs/1905.10437). 

**Data:** SILSO, World Data Center - Sunspot Number and Long-Term Solar Observations, Royal Observatory of Belgium, 1818 - 2021. https://wwwbis.sidc.be/silso/datafiles.

**Code:** TensorFlow (Python) implementation of N-BEATS model for univariate time series forecasting. https://github.com/flaviagiammarino/nbeats-tensorflow.

## 1. Set Up

Install the dependencies.

In [1]:
!pip install pandas numpy tensorflow plotly optuna

     |████████████████████████████████| 308 kB 4.5 MB/s 
     |████████████████████████████████| 209 kB 27.1 MB/s 
     |████████████████████████████████| 80 kB 3.7 MB/s 
     |████████████████████████████████| 75 kB 2.2 MB/s 
     |████████████████████████████████| 112 kB 57.8 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 148 kB 55.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=98a9799f9aec6bf11a029463217e819d77084aabe1e94a56c7bdeceed64c63a9
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


Clone the repository.

In [2]:
!pip install github-clone

In [3]:
!ghclone https://github.com/flaviagiammarino/nbeats-tensorflow/tree/main/nbeats_tensorflow

Cloning into 'nbeats_tensorflow'...
done.


Import the libraries.

In [4]:
import optuna
import logging
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go

In [5]:
from nbeats_tensorflow.model import NBeats

Set the logging level.

In [6]:
tf.get_logger().setLevel(logging.ERROR)
optuna.logging.set_verbosity(optuna.logging.INFO)

Fix the random seeds.

In [7]:
TENSORFLOW_SEED = 0
OPTUNA_SEED = 0

## 2. Data

Download the data.

In [8]:
df = pd.read_csv('https://wdc-silso-daily-sunspot-number.s3.eu-west-2.amazonaws.com/SN_d_tot_V2.0.csv', sep=';', header=None, usecols=[0, 1, 2, 4], names=['year', 'month', 'day', 'y'])

In [9]:
df

,year,month,day,y
0,1818,1,1,-1
1,1818,1,2,-1
2,1818,1,3,-1
3,1818,1,4,-1
4,1818,1,5,-1
...,...,...,...,...
74444,2021,10,27,104
74445,2021,10,28,100
74446,2021,10,29,85
74447,2021,10,30,78


Fill the missing values with zero.

In [10]:
df.loc[df['y'] == -1, 'y'] = 0

Generate the timestamps.

In [11]:
df['timestamp'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-' + df['day'].astype(str))

Drop the unnecessary columns.

In [12]:
df = df[['timestamp', 'y']]

Downsample the data from daily to monthly.

In [13]:
df = df.set_index('timestamp').resample('M')['y'].mean().reset_index()

In [14]:
df

,timestamp,y
0,1818-01-31,15.000000
1,1818-02-28,18.714286
2,1818-03-31,19.129032
3,1818-04-30,40.266667
4,1818-05-31,71.354839
...,...,...
2441,2021-06-30,24.966667
2442,2021-07-31,34.387097
2443,2021-08-31,22.387097
2444,2021-09-30,51.500000


Plot the data.

In [15]:
layout = dict(
    width=800,
    height=400,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, b=20, l=20, r=20),
    xaxis=dict(
        type='date',
        nticks=20,
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
    ),
    yaxis=dict(
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
        zeroline=False,
    ),
)

data = go.Scatter(
    x=df['timestamp'],
    y=df['y'],
    mode='lines',
    line=dict(
        color='#b3b3b3',
        width=0.5
    )
)

fig = go.Figure(data=data, layout=layout)

fig.show()

## 3. Model

Define the forecasting horizon.

In [16]:
forecast_period = 12 * 10 

Define a function for generating the forecasts and backcasts based on a set of hyperparameters.

In [17]:
def nn(target, forecast_period, params):
    
    '''
    Parameters:
    _______________________________
    target: np.array, pd.Series, list.
        Time series.
        
    forecast_period: int.
        Forecasting horizon.
    
    params: dict.
        Model parameters.
    
    Returns:
    _______________________________
    results: pd.DataFrame
        Forecast and backcast.
    '''

    tf.random.set_seed(TENSORFLOW_SEED)

    model = NBeats(
        target,
        forecast_period=forecast_period,
        lookback_period=params['multiplier'] * forecast_period,
        stacks=['trend', 'seasonality'],
        num_trend_coefficients=params['num_trend_coefficients'],
        num_seasonal_coefficients=params['num_seasonal_coefficients'],
        hidden_units=params['hidden_units'],
        num_blocks_per_stack=params['num_blocks_per_stack'],
        share_weights=params['share_weights'],
        share_coefficients=params['share_coefficients'],
    )

    model.fit(
        learning_rate=params['learning_rate'],
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        backcast_loss_weight=params['backcast_loss_weight'],
        loss='mae',
        validation_split=0,
        verbose=False
    )

    results = model.forecast(return_backcast=True)

    return results

Tune the hyperparameters.

In [18]:
# Split the data into training and validation, set aside the last sequence for testing.
y_train = df['y'].iloc[: - 2 * forecast_period]
y_valid = df['y'].iloc[- 2 * forecast_period: - forecast_period] 

# Define the objective function.
def objective(trial):

    # Sample the hyperparameters.
    params = {
        'multiplier': trial.suggest_int('multiplier', 2, 7),
        'num_trend_coefficients': trial.suggest_int('num_trend_coefficients', 2, 4),
        'num_seasonal_coefficients': trial.suggest_int('num_seasonal_coefficients', 2, 8),
        'hidden_units': trial.suggest_int('hidden_units', 10, 100, step=10),
        'num_blocks_per_stack': trial.suggest_int('num_blocks_per_stack', 1, 3),
        'share_weights': trial.suggest_categorical('share_weights', [True, False]),
        'share_coefficients': trial.suggest_categorical('share_coefficients', [True, False]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.01),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128, 256]),
        'epochs': trial.suggest_int('epochs', 100, 300, step=100),
        'backcast_loss_weight': trial.suggest_float('backcast_loss_weight', 0, 0.5, step=0.1),   
    }

    # Generate the forecast and backcast.
    df_pred = nn(
        target=y_train, 
        forecast_period=forecast_period, 
        params=params
    ) 

    # Extract the forecast.
    y_pred = df_pred['forecast'].iloc[- forecast_period:].values

    # Calculate the forecasting error.
    return np.mean(np.abs(y_valid - y_pred))

# Minimize the objective function.
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(OPTUNA_SEED))
study.optimize(objective, n_trials=50)

# Extract the best parameters.
best_params = study.best_params

[I 2021-11-12 19:33:43,848] A new study created in memory with name: no-name-84043c73-5376-41d3-8d93-8ae7457ed78a
[I 2021-11-12 19:34:02,802] Trial 0 finished with value: 57.893862913555246 and parameters: {'multiplier': 5, 'num_trend_coefficients': 4, 'num_seasonal_coefficients': 6, 'hidden_units': 60, 'num_blocks_per_stack': 2, 'share_weights': True, 'share_coefficients': False, 'learning_rate': 0.0024179177243329477, 'batch_size': 256, 'epochs': 100, 'backcast_loss_weight': 0.0}. Best is trial 0 with value: 57.893862913555246.
[I 2021-11-12 19:34:42,980] Trial 1 finished with value: 32.85572568821898 and parameters: {'multiplier': 2, 'num_trend_coefficients': 4, 'num_seasonal_coefficients': 7, 'hidden_units': 90, 'num_blocks_per_stack': 3, 'share_weights': True, 'share_coefficients': True, 'learning_rate': 0.00436436456821403, 'batch_size': 64, 'epochs': 100, 'backcast_loss_weight': 0.4}. Best is trial 1 with value: 32.85572568821898.
[I 2021-11-12 19:35:24,749] Trial 2 finished wit

In [19]:
best_params

{'backcast_loss_weight': 0.5,
 'batch_size': 256,
 'epochs': 300,
 'hidden_units': 100,
 'learning_rate': 0.0017676600857316625,
 'multiplier': 2,
 'num_blocks_per_stack': 1,
 'num_seasonal_coefficients': 5,
 'num_trend_coefficients': 4,
 'share_coefficients': False,
 'share_weights': True}

Generate the test set predictions.

In [20]:
df_test = nn(
    target=df['y'].iloc[: - forecast_period], 
    forecast_period=forecast_period, 
    params=best_params
) 

Add the timestamps to the output data frame.

In [21]:
df_test['time_idx'] = df['timestamp']

In [22]:
df_test.iloc[- 2 * forecast_period:]

,time_idx,actual,forecast,backcast
2206,2001-11-30,176.566667,NaN,176.902740
2207,2001-12-31,213.419355,NaN,213.749298
2208,2002-01-31,184.645161,NaN,184.968475
2209,2002-02-28,170.178571,NaN,170.494781
2210,2002-03-31,147.096774,NaN,147.405319
...,...,...,...,...
2441,2021-06-30,NaN,0.308618,NaN
2442,2021-07-31,NaN,1.049440,NaN
2443,2021-08-31,NaN,1.885758,NaN
2444,2021-09-30,NaN,2.817844,NaN


Plot the test set predictions.

In [23]:
layout = dict(
    width=800,
    height=425,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, b=20, l=20, r=20),
    legend=dict(
        font=dict(
            color='#3a3a3a',
            size=10,
        ),
        orientation='h',
        x=0.0,
        y=1.1,
    ),
    xaxis=dict(
        type='date',
        nticks=20,
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
    ),
    yaxis=dict(
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
        zeroline=False,
    ),
)

data = []

data.append(
    go.Scatter(
        x=df['timestamp'],
        y=df['y'],
        name='Actual',
        mode='lines',
        line=dict(
            color='#b3b3b3',
            width=0.5
        )
    )
)

data.append(
    go.Scatter(
        x=df_test['time_idx'],
        y=df_test['forecast'],
        name='Forecast',
        mode='lines',
        line=dict(
            color='#0550ae',
            width=1
          )
      )
)

data.append(
    go.Scatter(
        x=df_test['time_idx'],
        y=df_test['backcast'],
        name='Backcast',
        mode='lines',
        line=dict(
            color='#8250df',
            width=1
        )
    )
)

fig = go.Figure(data=data, layout=layout)

fig.show()

## 4. Results

Fit the model using all the data.

In [24]:
df_future = nn(
    target=df['y'], 
    forecast_period=forecast_period, 
    params=best_params
)

Add the timestamps to the output data frame.

In [25]:
df_future['time_idx'].iloc[:- forecast_period] = df['timestamp']
df_future['time_idx'].iloc[- forecast_period:] = pd.date_range(start=df['timestamp'].iloc[-1], periods=1 + forecast_period, freq='M')[1:]

In [26]:
df_future.iloc[- 2 * forecast_period:]

,time_idx,actual,forecast,backcast
2326,2011-11-30 00:00:00,139.066667,NaN,138.281693
2327,2011-12-31 00:00:00,109.290323,NaN,108.514587
2328,2012-01-31 00:00:00,94.419355,NaN,93.652260
2329,2012-02-29 00:00:00,47.758621,NaN,46.999592
2330,2012-03-31 00:00:00,86.645161,NaN,85.893631
...,...,...,...,...
2561,2031-06-30 00:00:00,NaN,1.436062,NaN
2562,2031-07-31 00:00:00,NaN,0.953183,NaN
2563,2031-08-31 00:00:00,NaN,0.534907,NaN
2564,2031-09-30 00:00:00,NaN,0.186163,NaN


Plot the out of sample forecasts.

In [27]:
layout = dict(
    width=800,
    height=425,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, b=20, l=20, r=20),
    legend=dict(
        font=dict(
            color='#3a3a3a',
            size=10,
        ),
        orientation='h',
        x=0.0,
        y=1.1,
    ),
    xaxis=dict(
        type='date',
        nticks=20,
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
    ),
    yaxis=dict(
        tickfont=dict(
            color='#3a3a3a',
            size=10,
        ),
        linecolor='#d9d9d9',
        mirror=True,
        showgrid=False,
        zeroline=False,
    ),
)

data = []

data.append(
    go.Scatter(
        x=df['timestamp'],
        y=df['y'],
        name='Actual',
        mode='lines',
        line=dict(
            color='#b3b3b3',
            width=0.5
        )
    )
)

data.append(
    go.Scatter(
        x=df_future['time_idx'],
        y=df_future['forecast'],
        name='Forecast',
        mode='lines',
        line=dict(
            color='#0550ae',
            width=1
          )
      )
)

data.append(
    go.Scatter(
        x=df_future['time_idx'],
        y=df_future['backcast'],
        name='Backcast',
        mode='lines',
        line=dict(
            color='#8250df',
            width=1
        )
    )
)

fig = go.Figure(data=data, layout=layout)

fig.show()